# PyTorch Workflow

In [2]:
learning = {1: "Data (prepare and load",
            2: "build model",
            3: "fitting the model to data (training)",
            4: "making prediction and evaluating the model (inference)",
            5: "saving and loading a model",
            6: "putting it all together"}
learning

{1: 'Data (prepare and load',
 2: 'build model',
 3: 'fitting the model to data (training)',
 4: 'making prediction and evaluating the model (inference)',
 5: 'saving and loading a model',
 6: 'putting it all together'}

In [3]:
import torch
from torch import nn
import matplotlib.pyplot as plt

# Checking versions
torch.__version__

'2.9.1+cu128'

# 1. Data (preparing and loading)

Data can be almost anything… in machine learning.

- Excel spreadsheet  
- Images of any kind  
- Videos (YouTube has lots of data…)  
- Audio like songs or podcasts  
- DNA  
- Text  

## Machine learning is a game of two parts:

1. Get data into a numerical representation.  
2. Build a model to learn patterns in that numerical representation.


In [27]:
weight = 0.7
bias = 0.3

start = 0
end = 1
step = 0.02

X = torch.arange(start,end,step).unsqueeze(dim=1)
y = weight * X + bias

X[:10], y[:10]

(tensor([[0.0000],
         [0.0200],
         [0.0400],
         [0.0600],
         [0.0800],
         [0.1000],
         [0.1200],
         [0.1400],
         [0.1600],
         [0.1800]]),
 tensor([[0.3000],
         [0.3140],
         [0.3280],
         [0.3420],
         [0.3560],
         [0.3700],
         [0.3840],
         [0.3980],
         [0.4120],
         [0.4260]]))

In [28]:
len(X), len(y)

(50, 50)

### Splitting the data into training and test set

In [32]:
train_split = int(0.8 * len(X))

X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]

len(X_train), len(y_train), len(X_test), len(y_test)

(40, 40, 10, 10)